In [1]:
import pandas as pd
import pickle
import nltk
from nltk.chunk import ne_chunk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.tokenize import sent_tokenize
import re
from nltk.corpus import stopwords
import string
from geopy.geocoders import Nominatim

In [3]:
with open("0to1_sentences_w_units_terms_nolocation.pkl", 'rb') as picklefile:
   df = pickle.load(picklefile)

In [4]:
df.head()

,_id,authors,date_x,image,keywords,summary,text,url,valid,decode_text,error,date_y,language,doc_id,sent_w_num,sent_w_num_p,sent_w_num_h,sent_num_p_t,sent_num_h_t
0,58c94fc7d897da23f9f5a246,[],NaT,,"[somalilandpress, meant, appear, javascript, r...",In order for you to see this page as it is mea...,Oops! It appears that you have disabled your J...,http://www.somalilandpress.com/iom-supports-dr...,True,Oops! It appears that you have disabled your J...,all good,20160512000000,en,0,[],[],[],[],[]
1,58c94fc9d897da23f9f5a247,[],NaT,,"[requested, updated, return, unavailable, nort...",You may have followed an outdated link or have...,The page you requested is currently unavailabl...,http://www.9and10news.com/story/33699640/syria...,True,The page you requested is currently unavailabl...,all good,20161114080000,en,1,[],[],[],[],[]
2,58c94fcad897da23f9f5a248,[],2016-12-19 19:32:57,https://c.tribune.com.pk/2016/12/1268397-saudi...,"[defence, bombs, sold, coalition, british, war...","“However, Saudi Arabia has now confirmed it wi...","The war has killed more than 10,000 people and...",http://tribune.com.pk/story/1268397/saudi-led-...,True,"The war has killed more than 10,000 people and...",all good,20161219200000,en,2,"[The war has killed more than 10,000 people an...","[The war has killed more than 10,000 people an...",[],[],[]
3,58c94fd4d897da23f9f5a249,[Reuters Staff Writers News Corp Australia Net...,2016-12-20 02:51:00,http://cdn.newsapi.com.au/image/v1/0152297c79d...,"[homes, oil, followed, quake, deadly, coast, e...",That quake flattened homes and buildings up an...,A 5.7 MAGNITUDE earthquake followed by 37 afte...,http://www.couriermail.com.au/news/world/ecuad...,True,A 5.7 MAGNITUDE earthquake followed by 37 afte...,all good,20161219200000,en,3,[A 5.7 MAGNITUDE earthquake followed by 37 aft...,[Three people died and 47 were injured as a re...,[At least 700 families had to leave their home...,[],[]
4,58c94fd6d897da23f9f5a24c,[],NaT,http://ftpcontent.worldnow.com/wncustom/custom...,"[requested, updated, removed, return, unavaila...",You may have followed an outdated link or have...,The page you requested is currently unavailabl...,http://www.nbc-2.com/story/32756524/the-latest...,True,The page you requested is currently unavailabl...,all good,20160815040000,en,4,[],[],[],[],[]


In [5]:
df.columns

Index([         u'_id',      u'authors',       u'date_x',        u'image',
           u'keywords',      u'summary',         u'text',          u'url',
              u'valid',  u'decode_text',        u'error',       u'date_y',
           u'language',       u'doc_id',   u'sent_w_num', u'sent_w_num_p',
       u'sent_w_num_h', u'sent_num_p_t', u'sent_num_h_t'],
      dtype='object')

## People facts

In [8]:
df_people = df.set_index(['doc_id', 'date_x', 'date_y', 'image', 'url',
                          'decode_text'])['sent_num_p_t'].apply(pd.Series).stack()

df_people = df_people.reset_index()

df_people.columns = ['doc_id', 'date_x', 'date_y', 'image', 'url', 
                     'decode_text' ,'sentence_number','sen_num_p_t']

df_people.head()

,doc_id,date_x,date_y,image,url,decode_text,sentence_number,sen_num_p_t
0,10,2016-01-28 19:47:17,20160128160000,http://i.dailymail.co.uk/i/pix/2016/01/28/arti...,http://www.dailymail.co.uk/wires/pa/article-34...,Storm Gertrude set to sweep in with 90mph gust...,0,Around 600 people were evacuated from their ho...
1,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,0,"Earlier on Monday, the NDRF said it had rescue..."
2,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,1,"""An NDRF team pre-positioned in Sikkim conduct..."
3,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,2,"""So far, the NDRF teams have evacuated more th..."
4,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,3,The NDRF said 11 flood rescue teams rescued 27...


In [9]:
df_people.shape

(43778, 8)

In [10]:
def word_tokenize_filter(sentence):
    words = word_tokenize(sentence)
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    filtered_words = [c for c in filtered_words if c not in string.punctuation]
    return filtered_words

In [11]:
df_people['w_tokens'] = df_people.sen_num_p_t.apply(word_tokenize_filter)
df_people.head()

,doc_id,date_x,date_y,image,url,decode_text,sentence_number,sen_num_p_t,w_tokens
0,10,2016-01-28 19:47:17,20160128160000,http://i.dailymail.co.uk/i/pix/2016/01/28/arti...,http://www.dailymail.co.uk/wires/pa/article-34...,Storm Gertrude set to sweep in with 90mph gust...,0,Around 600 people were evacuated from their ho...,"[Around, 600, people, evacuated, homes, Scotti..."
1,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,0,"Earlier on Monday, the NDRF said it had rescue...","[Earlier, Monday, NDRF, said, rescued, 26,000,..."
2,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,1,"""An NDRF team pre-positioned in Sikkim conduct...","[``, An, NDRF, team, pre-positioned, Sikkim, c..."
3,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,2,"""So far, the NDRF teams have evacuated more th...","[``, So, far, NDRF, teams, evacuated, 26,400, ..."
4,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,3,The NDRF said 11 flood rescue teams rescued 27...,"[The, NDRF, said, 11, flood, rescue, teams, re..."


In [12]:
grammar = r'''Counts: {<NN>?<RB>?<JJ>?<CD><CD>?<JJ>*<NNS>*<RB>*<JJ>*<NNS>*<NNP>*<JJ>*<VB.?><JJ>?}'''

def chunk_finder(word_list):
    tags = pos_tag(word_list)
    chunkParser = nltk.RegexpParser(grammar)
    chunked = chunkParser.parse(tags)
    holychunk = [chunk for chunk in chunked if type(chunk) == nltk.tree.Tree]
    sentence_facts = []
    for holyc in holychunk:
        fact_list = []
        for specialtuples in holyc:
            if specialtuples[1] == "CD":
                number = specialtuples[0]
                fact_list.append(number)
            elif specialtuples[1] == "NNS":
                unit = specialtuples[0]
                fact_list.append(unit)
            elif specialtuples[1] == "VBN" or specialtuples[1] == "JJ" or specialtuples[1] == "VBD":
                verb = specialtuples[0]
                fact_list.append(verb)
            else:
                continue
        sentence_facts.append(fact_list)
    return sentence_facts


In [13]:
df_people['chunks'] = df_people.w_tokens.apply(chunk_finder)

In [15]:
df_people.head()

,doc_id,date_x,date_y,image,url,decode_text,sentence_number,sen_num_p_t,w_tokens,chunks
0,10,2016-01-28 19:47:17,20160128160000,http://i.dailymail.co.uk/i/pix/2016/01/28/arti...,http://www.dailymail.co.uk/wires/pa/article-34...,Storm Gertrude set to sweep in with 90mph gust...,0,Around 600 people were evacuated from their ho...,"[Around, 600, people, evacuated, homes, Scotti...","[[600, people, evacuated]]"
1,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,0,"Earlier on Monday, the NDRF said it had rescue...","[Earlier, Monday, NDRF, said, rescued, 26,000,...",[]
2,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,1,"""An NDRF team pre-positioned in Sikkim conduct...","[``, An, NDRF, team, pre-positioned, Sikkim, c...",[]
3,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,2,"""So far, the NDRF teams have evacuated more th...","[``, So, far, NDRF, teams, evacuated, 26,400, ...",[]
4,11,2016-08-23 03:20:59,20160823040000,http://s2.firstpost.in/wp-content/uploads/2016...,http://www.firstpost.com/india/flood-situation...,New Delhi: The Centre on Monday night rushed 1...,3,The NDRF said 11 flood rescue teams rescued 27...,"[The, NDRF, said, 11, flood, rescue, teams, re...",[]


In [16]:
df_people_chunks = df_people.set_index(['doc_id', 'date_x', 'date_y', 'image', 'url', 
                     'decode_text' ,'sentence_number','sen_num_p_t'])['chunks'].apply(pd.Series).stack()

df_people_chunks = df_people_chunks.reset_index()

df_people_chunks.columns = ['doc_id', 'date_x', 'date_y', 'image', 'url', 
                     'decode_text' ,'sentence_number','sen_num_p_t','chunk_num','chunks']

df_people_chunks.head()

,doc_id,date_x,date_y,image,url,decode_text,sentence_number,sen_num_p_t,chunk_num,chunks
0,10,2016-01-28 19:47:17,20160128160000,http://i.dailymail.co.uk/i/pix/2016/01/28/arti...,http://www.dailymail.co.uk/wires/pa/article-34...,Storm Gertrude set to sweep in with 90mph gust...,0,Around 600 people were evacuated from their ho...,0,"[600, people, evacuated]"
1,12,2016-10-10 21:28:02,20161010080000,http://cmgfeeds.cmgdigital.com/photo/2016/10/1...,http://www.kiro7.com/news/national/aftereffect...,"by: EMERY P. DALESIO and MARTHA WAGGONER, Asso...",1,"About 1,500 people had to be rescued early Mon...",0,"[1,500, people, rescued, early]"
2,21,2016-06-04 00:36:56,20160603040000,http://sharing.wkbw.com/sharescnn/photo/2016/0...,http://www.wkbw.com/news/national/three-soldie...,"FORT HOOD, Texas (AP) Army teams and emergenc...",0,"""With the rain that's predicted, that's not go...",0,"[700, people, evacuated]"
3,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,0,Syrias five-year conflict has killed more than...,0,"[270,000, people, forced]"
4,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,1,The shutdown has led to a bottleneck at the Gr...,0,"[11,500, people, stranded, squalid]"


In [19]:
people_units = ['people', 'peoples', 'persons', 'individuals', 'children', 'inhabitants',
                'residents', 'migrants','civilians', 'children', 'villagers', 'citizens', 'refugees', 'minors',
               'evacuees', 'kids', 'idps', 'beneficiaries', 'tenants']

In [20]:
# NEED TO FIX NUMBERS THAT ARE WRITTEN!!!!

def people_unit(chunklist):
    numbers = []
    for word in people_units: 
        for e in chunklist:
            if word == e:
                tags = pos_tag(chunklist)
                for tag in tags:
                    if tag[1] == "CD":
                        numbers.append(tag[0])
    if len(numbers) == 0 or len(numbers) == 1:
        return numbers
    else:
        try:
            if numbers[1] == 'million':
                return int(numbers[0]) * 1000000
            elif numbers[1] == 'thousand':
                return int(numbers[0]) * 1000
            elif numbers[1] == 'hundred':
                return int(numbers[0]) * 100 
        except:
            pass
        

In [21]:
df_people_chunks['people_numbers'] = df_people_chunks.chunks.apply(people_unit)

In [22]:
def get_number(x):
    try:
        number = str(x[0])
        number = number.replace(",","")
        number = int(number)
        return number
    except:
        pass

In [23]:
df_people_chunks['final_number'] = df_people_chunks.people_numbers.apply(get_number)

In [24]:
df_people_chunks.head()

,doc_id,date_x,date_y,image,url,decode_text,sentence_number,sen_num_p_t,chunk_num,chunks,people_numbers,final_number
0,10,2016-01-28 19:47:17,20160128160000,http://i.dailymail.co.uk/i/pix/2016/01/28/arti...,http://www.dailymail.co.uk/wires/pa/article-34...,Storm Gertrude set to sweep in with 90mph gust...,0,Around 600 people were evacuated from their ho...,0,"[600, people, evacuated]",[600],600.0
1,12,2016-10-10 21:28:02,20161010080000,http://cmgfeeds.cmgdigital.com/photo/2016/10/1...,http://www.kiro7.com/news/national/aftereffect...,"by: EMERY P. DALESIO and MARTHA WAGGONER, Asso...",1,"About 1,500 people had to be rescued early Mon...",0,"[1,500, people, rescued, early]","[1,500]",1500.0
2,21,2016-06-04 00:36:56,20160603040000,http://sharing.wkbw.com/sharescnn/photo/2016/0...,http://www.wkbw.com/news/national/three-soldie...,"FORT HOOD, Texas (AP) Army teams and emergenc...",0,"""With the rain that's predicted, that's not go...",0,"[700, people, evacuated]",[700],700.0
3,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,0,Syrias five-year conflict has killed more than...,0,"[270,000, people, forced]","[270,000]",270000.0
4,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,1,The shutdown has led to a bottleneck at the Gr...,0,"[11,500, people, stranded, squalid]","[11,500]",11500.0


In [25]:
final_people = df_people_chunks[df_people_chunks.final_number > 0]

In [26]:
final_people.shape

(17605, 12)

In [29]:
displacement_terms = ['airlifted', 'affected', 'bused',' bussed', 'displace', 'displaced', 'displaces' 'destroyed', 'evacuate', 'evacuated', 
                    'evacuating', 'evicted', 'ferried' ,'forced', 'flee','fled', 'fleeing', 'homeless', 'housed', 'moved', 'relief', 
         'relocate', 'relocated', 'rescued', 'reocuppied', 'repatriated' 'sheltered', 'shelters', 'sheltering', 'submerged', 'transferred' 
                      'transport', 'transported', 'uprooted', 'uninhabitable', 'washed']

In [30]:
def term_unit(chunklist):
    verb = []
    for word in displacement_terms: 
        for e in chunklist:
            if word == e:
                tags = pos_tag(chunklist)
                for tag in tags:
                    if tag[1] == "VB" or tag[1] == "VBD" or tag[1] == "VBG" or tag[1] == "VBN" or \
                    tag[1] == "VBP" or tag[1] == "VBZ": # or tag[1] == "JJ":
                        verb.append(tag[0])
    return verb

In [31]:
final_people['verb'] = final_people.chunks.apply(term_unit)

/Users/lorenadelaparralanda/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [32]:
final_people.head()

,doc_id,date_x,date_y,image,url,decode_text,sentence_number,sen_num_p_t,chunk_num,chunks,people_numbers,final_number,verb
0,10,2016-01-28 19:47:17,20160128160000,http://i.dailymail.co.uk/i/pix/2016/01/28/arti...,http://www.dailymail.co.uk/wires/pa/article-34...,Storm Gertrude set to sweep in with 90mph gust...,0,Around 600 people were evacuated from their ho...,0,"[600, people, evacuated]",[600],600.0,[evacuated]
1,12,2016-10-10 21:28:02,20161010080000,http://cmgfeeds.cmgdigital.com/photo/2016/10/1...,http://www.kiro7.com/news/national/aftereffect...,"by: EMERY P. DALESIO and MARTHA WAGGONER, Asso...",1,"About 1,500 people had to be rescued early Mon...",0,"[1,500, people, rescued, early]","[1,500]",1500.0,[rescued]
2,21,2016-06-04 00:36:56,20160603040000,http://sharing.wkbw.com/sharescnn/photo/2016/0...,http://www.wkbw.com/news/national/three-soldie...,"FORT HOOD, Texas (AP) Army teams and emergenc...",0,"""With the rain that's predicted, that's not go...",0,"[700, people, evacuated]",[700],700.0,[evacuated]
3,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,0,Syrias five-year conflict has killed more than...,0,"[270,000, people, forced]","[270,000]",270000.0,[forced]
4,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,1,The shutdown has led to a bottleneck at the Gr...,0,"[11,500, people, stranded, squalid]","[11,500]",11500.0,[]


In [33]:
def get_verb(x):
    try:
        return str(x[0])
    except:
        pass

In [34]:
final_people['final_verb'] = final_people.verb.apply(get_verb)

/Users/lorenadelaparralanda/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [35]:
final_people_displaced = final_people[final_people.verb != None]

In [36]:
final_people_displaced.shape

(17605, 14)

In [37]:
final_people_displaced.head()

,doc_id,date_x,date_y,image,url,decode_text,sentence_number,sen_num_p_t,chunk_num,chunks,people_numbers,final_number,verb,final_verb
0,10,2016-01-28 19:47:17,20160128160000,http://i.dailymail.co.uk/i/pix/2016/01/28/arti...,http://www.dailymail.co.uk/wires/pa/article-34...,Storm Gertrude set to sweep in with 90mph gust...,0,Around 600 people were evacuated from their ho...,0,"[600, people, evacuated]",[600],600.0,[evacuated],evacuated
1,12,2016-10-10 21:28:02,20161010080000,http://cmgfeeds.cmgdigital.com/photo/2016/10/1...,http://www.kiro7.com/news/national/aftereffect...,"by: EMERY P. DALESIO and MARTHA WAGGONER, Asso...",1,"About 1,500 people had to be rescued early Mon...",0,"[1,500, people, rescued, early]","[1,500]",1500.0,[rescued],rescued
2,21,2016-06-04 00:36:56,20160603040000,http://sharing.wkbw.com/sharescnn/photo/2016/0...,http://www.wkbw.com/news/national/three-soldie...,"FORT HOOD, Texas (AP) Army teams and emergenc...",0,"""With the rain that's predicted, that's not go...",0,"[700, people, evacuated]",[700],700.0,[evacuated],evacuated
3,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,0,Syrias five-year conflict has killed more than...,0,"[270,000, people, forced]","[270,000]",270000.0,[forced],forced
4,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,1,The shutdown has led to a bottleneck at the Gr...,0,"[11,500, people, stranded, squalid]","[11,500]",11500.0,[],None


In [38]:
with open("0to1_numbersandverbs.pkl", "wb") as picklefile:
    pickle.dump(final_people_displaced, picklefile)

## Conquer location

In [39]:
def word_tokenize_filter(sentence):
    words = word_tokenize(sentence)
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    filtered_words = [c for c in filtered_words if c not in string.punctuation]
    return filtered_words

In [40]:
final_people_displaced['doc_word_tokens'] = final_people_displaced.decode_text.apply(word_tokenize_filter)

In [41]:
final_people_displaced.head()

,doc_id,date_x,date_y,image,url,decode_text,sentence_number,sen_num_p_t,chunk_num,chunks,people_numbers,final_number,verb,final_verb,doc_word_tokens
0,10,2016-01-28 19:47:17,20160128160000,http://i.dailymail.co.uk/i/pix/2016/01/28/arti...,http://www.dailymail.co.uk/wires/pa/article-34...,Storm Gertrude set to sweep in with 90mph gust...,0,Around 600 people were evacuated from their ho...,0,"[600, people, evacuated]",[600],600.0,[evacuated],evacuated,"[Storm, Gertrude, set, sweep, 90mph, gusts, St..."
1,12,2016-10-10 21:28:02,20161010080000,http://cmgfeeds.cmgdigital.com/photo/2016/10/1...,http://www.kiro7.com/news/national/aftereffect...,"by: EMERY P. DALESIO and MARTHA WAGGONER, Asso...",1,"About 1,500 people had to be rescued early Mon...",0,"[1,500, people, rescued, early]","[1,500]",1500.0,[rescued],rescued,"[EMERY, P., DALESIO, MARTHA, WAGGONER, Associa..."
2,21,2016-06-04 00:36:56,20160603040000,http://sharing.wkbw.com/sharescnn/photo/2016/0...,http://www.wkbw.com/news/national/three-soldie...,"FORT HOOD, Texas (AP) Army teams and emergenc...",0,"""With the rain that's predicted, that's not go...",0,"[700, people, evacuated]",[700],700.0,[evacuated],evacuated,"[FORT, HOOD, Texas, AP, Army, teams, emergency..."
3,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,0,Syrias five-year conflict has killed more than...,0,"[270,000, people, forced]","[270,000]",270000.0,[forced],forced,"[Pope, Francis, Sunday, spoke, rejection, refu..."
4,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,1,The shutdown has led to a bottleneck at the Gr...,0,"[11,500, people, stranded, squalid]","[11,500]",11500.0,[],None,"[Pope, Francis, Sunday, spoke, rejection, refu..."


In [42]:
import pycountry

def one_locations(locations):
    countries = []
    for location in locations:
        try:
            country = pycountry.countries.get(name=location)
            if country.numeric > 0:
                country = country.name
                countries.append(country)
        except:
            pass
    return countries

In [43]:
final_people_displaced['countries'] = final_people_displaced.doc_word_tokens.apply(one_locations)

In [44]:
final_people_displaced.head()

,doc_id,date_x,date_y,image,url,decode_text,sentence_number,sen_num_p_t,chunk_num,chunks,people_numbers,final_number,verb,final_verb,doc_word_tokens,countries
0,10,2016-01-28 19:47:17,20160128160000,http://i.dailymail.co.uk/i/pix/2016/01/28/arti...,http://www.dailymail.co.uk/wires/pa/article-34...,Storm Gertrude set to sweep in with 90mph gust...,0,Around 600 people were evacuated from their ho...,0,"[600, people, evacuated]",[600],600.0,[evacuated],evacuated,"[Storm, Gertrude, set, sweep, 90mph, gusts, St...","[Ireland, Ireland]"
1,12,2016-10-10 21:28:02,20161010080000,http://cmgfeeds.cmgdigital.com/photo/2016/10/1...,http://www.kiro7.com/news/national/aftereffect...,"by: EMERY P. DALESIO and MARTHA WAGGONER, Asso...",1,"About 1,500 people had to be rescued early Mon...",0,"[1,500, people, rescued, early]","[1,500]",1500.0,[rescued],rescued,"[EMERY, P., DALESIO, MARTHA, WAGGONER, Associa...","[Haiti, Georgia, Georgia, Georgia]"
2,21,2016-06-04 00:36:56,20160603040000,http://sharing.wkbw.com/sharescnn/photo/2016/0...,http://www.wkbw.com/news/national/three-soldie...,"FORT HOOD, Texas (AP) Army teams and emergenc...",0,"""With the rain that's predicted, that's not go...",0,"[700, people, evacuated]",[700],700.0,[evacuated],evacuated,"[FORT, HOOD, Texas, AP, Army, teams, emergency...",[Singapore]
3,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,0,Syrias five-year conflict has killed more than...,0,"[270,000, people, forced]","[270,000]",270000.0,[forced],forced,"[Pope, Francis, Sunday, spoke, rejection, refu...","[Turkey, Turkey, Turkey, Turkey, Greece, Belgi..."
4,25,NaT,20160327200000,http://www.ekathimerini.com/resources/2016-03/...,http://www.ekathimerini.com/207360/article/eka...,Pope Francis on Sunday spoke out against the r...,1,The shutdown has led to a bottleneck at the Gr...,0,"[11,500, people, stranded, squalid]","[11,500]",11500.0,[],None,"[Pope, Francis, Sunday, spoke, rejection, refu...","[Turkey, Turkey, Turkey, Turkey, Greece, Belgi..."


In [45]:
import collections

def country(countries):
    try:
        counter = collections.Counter(countries)
        country = counter.most_common(1)[0][0]
        one_country = country.encode("ascii","ignore")
        return one_country
    except:
        pass

In [46]:
final_people_displaced['one_country'] = final_people_displaced.countries.apply(country)

In [47]:
with open("0to1_country_numbers_verb.pkl", "wb") as picklefile:
    pickle.dump(df, picklefile)

## Explore

In [48]:
final_people_displaced.groupby(['one_country'])['final_number'].sum()

one_country
Afghanistan     11288359.0
Albania              475.0
Algeria            27000.0
Angola            570250.0
Antarctica         70000.0
Argentina         568256.0
Armenia             2879.0
Australia        2226228.0
Austria           455470.0
Azerbaijan       4422392.0
Bahamas          5053098.0
Bahrain           523000.0
Bangladesh      16749091.0
Belarus           127000.0
Belgium            35756.0
Belize            572058.0
Benin            1472418.0
Bermuda              264.0
Bhutan            639038.0
Brazil            251651.0
Bulgaria            6644.0
Burundi          1804609.0
Cambodia            3000.0
Cameroon         3457960.0
Canada          42576548.0
Chad            19316654.0
Chile             262694.0
China           28407700.0
Colombia        87110846.0
Congo            5629821.0
                  ...     
Poland           1321514.0
Portugal           42061.0
Qatar            2081661.0
Romania              654.0
Rwanda           1191588.0
Samoa           

In [49]:
final_people_displaced[final_people_displaced['one_country'] == "Mexico"]

,doc_id,date_x,date_y,image,url,decode_text,sentence_number,sen_num_p_t,chunk_num,chunks,people_numbers,final_number,verb,final_verb,doc_word_tokens,countries,one_country
21,58,2016-08-15 22:40:11,20160815200000,http://news.kuwaittimes.net/website/wp-content...,http://news.kuwaittimes.net/website/flood-leav...,Flood leaves thousands homeless in Louisiana\n...,0,"Approximately 18,000 people have been rescued ...",0,"[18,000, people, rescued]","[18,000]",18000.0,[rescued],rescued,"[Flood, leaves, thousands, homeless, Louisiana...",[Mexico],Mexico
22,58,2016-08-15 22:40:11,20160815200000,http://news.kuwaittimes.net/website/wp-content...,http://news.kuwaittimes.net/website/flood-leav...,Flood leaves thousands homeless in Louisiana\n...,2,"As of Sunday morning, some 5,050 people were s...",0,"[5,050, people, parish]","[5,050]",5050.0,[],None,"[Flood, leaves, thousands, homeless, Louisiana...",[Mexico],Mexico
173,386,NaT,20160620053000,http://www.kob.com/kobtvimages/repository/2016...,http://www.KOB.com/new-mexico-news/the-latest-...,Horses and other animals are also being evacua...,1,Fire officials had initially said the entire c...,0,"[600, people]",[600],600.0,[],None,"[Horses, animals, also, evacuated, ranching, c...","[Mexico, Mexico, Mexico]",Mexico
225,508,NaT,20160621151500,http://static-24.sinclairstoryline.com/resourc...,http://nbc24.com/news/nation-world/2-big-wildf...,"A firefighter keeps watch a wildfire in Azusa,...",0,It surged to nearly 12 square miles and forced...,1,"[75, people]",[75],75.0,[],None,"[A, firefighter, keeps, watch, wildfire, Azusa...","[Mexico, Mexico, Mexico, Mexico, Mexico]",Mexico
400,841,2016-05-31 00:00:00,20160601044500,http://www.trbimg.com/img-574e5b4e/turbine/ct-...,http://www.dailypress.com/news/nationworld/ct-...,Residents of some rural southeastern Texas cou...,0,About 40 people were rescued Sunday and Monday...,0,"[40, people, rescued]",[40],40.0,[rescued],rescued,"[Residents, rural, southeastern, Texas, counti...",[Mexico],Mexico
796,1653,2016-08-18 20:00:12,20160818131500,https://cdn2.newsok.biz/cache/r960_blur-9d89b4...,http://newsok.com/the-latest-dogs-displaced-by...,"DENHAM SPRINGS, La. (AP) The Latest on floodi...",0,"The state estimated about 4,000 people remaine...",0,"[4,000, people, remained]","[4,000]",4000.0,[],None,"[DENHAM, SPRINGS, La, AP, The, Latest, floodin...",[Mexico],Mexico
798,1653,2016-08-18 20:00:12,20160818131500,https://cdn2.newsok.biz/cache/r960_blur-9d89b4...,http://newsok.com/the-latest-dogs-displaced-by...,"DENHAM SPRINGS, La. (AP) The Latest on floodi...",1,"Edwards said more than 40,000 homes were affec...",1,"[30,000, people, rescued]","[30,000]",30000.0,[rescued],rescued,"[DENHAM, SPRINGS, La, AP, The, Latest, floodin...",[Mexico],Mexico
955,1962,NaT,20160531144500,http://www.tampabay.com/resources/images/dti/r...,http://www.tampabay.com/news/weather/torrentia...,"FORT WORTH, Texas Residents of some rural sou...",0,Aerial photos taken Sunday showed large swaths...,0,"[1,000, people, evacuated]","[1,000]",1000.0,[evacuated],evacuated,"[FORT, WORTH, Texas, Residents, rural, southea...","[Mexico, Mexico, Mexico]",Mexico
956,1962,NaT,20160531144500,http://www.tampabay.com/resources/images/dti/r...,http://www.tampabay.com/news/weather/torrentia...,"FORT WORTH, Texas Residents of some rural sou...",1,About 40 people were rescued Sunday and Monday...,0,"[40, people, rescued]",[40],40.0,[rescued],rescued,"[FORT, WORTH, Texas, Residents, rural, southea...","[Mexico, Mexico, Mexico]",Mexico
1157,2352,2016-05-30 00:00:00,20160530141500,http://www.news1130.com/wp-content/blogs.dir/s...,http://www.news1130.com/2016/05/30/floods-in-t...,"FORT WORTH, Texas In a story May 30 about flo...",0,About 40 people were rescued from late Sunday ...,0,"[40, people, rescued, late]",[40],40.0,[rescued],rescued,"[FORT, WORTH, Texas, In, story, May, 30, flood...","[Mexico, Mexico, Georgia]",Mexico
